In [73]:
import os
from pathlib import Path
import xmlrpc.client
import pandas as pd
from sqlalchemy import create_engine

In [74]:
def api_params_func(test_db: bool = False) -> dict:

    api_url = os.environ.get('ODOO_URL_API')
    api_db = os.environ.get('ODOO_DB_API')
    api_test_db = os.environ.get('ODOO_DB_PRUEBA_API')
    api_username = os.environ.get('ODOO_USERNAME_API')
    api_clave = os.environ.get('ODOO_CLAVE_API')


    api_params = {}
    if test_db:
        api_params['api_db'] = api_test_db
    else:
        api_params['api_db'] = api_db


    common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
    uid = common.authenticate(api_params['api_db'], api_username, api_clave, {})
    models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')


    api_params['api_clave'] = api_clave
    api_params['api_uid'] = uid
    api_params['api_models'] = models

    return api_params

In [75]:
def search_costo_ventas_func(mes: int, tolerance: list) -> list[str]:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    if len(tolerance) != 2:
        raise Exception (f'La tolerancia debe ser una lista de sólo dos elementos')

    if type(tolerance[0]) != int or type(tolerance[1]) != int:
        raise Exception (f'Los elementos de la lista de tolerancia deben ser sólo números tipo enteros')
    
    param_dia_hr_ini = pd.Timestamp(2024, mes, 1) + pd.Timedelta(hours=7)
    param_dia_hr_fin = pd.Timestamp(2024, mes+1, 1) + pd.Timedelta(hours=7) - pd.Timedelta(seconds= 1)
    
    param_dia_hr_ini_tolerance = param_dia_hr_ini - pd.Timedelta(days = tolerance[0])
    param_dia_hr_fin_tolerance = param_dia_hr_fin + pd.Timedelta(days = tolerance[1])
    
    search_costo_ventas = [
        "&", "&",
            ("state", "in", ["purchase", "done"]),
            ("date_approve", ">=", param_dia_hr_ini_tolerance.strftime('%Y-%m-%d %H:%M:%S')),
            ("date_approve", "<=", param_dia_hr_fin_tolerance.strftime('%Y-%m-%d %H:%M:%S')),
        ]

    return search_costo_ventas

In [76]:
def mes_to_string_func(mes: int) -> str:
    
    if type(mes) != int or mes < 1 or mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {mes}')
    
    match mes:
        case 1:
            mes_to_string = 'enero'
        case 2:
            mes_to_string = 'febrero'
        case 3:
            mes_to_string = 'marzo'
        case 4:
            mes_to_string = 'abril'
        case 5:
            mes_to_string = 'mayo'
        case 6:
            mes_to_string = 'junio'
        case 7:
            mes_to_string = 'julio'
        case 8:
            mes_to_string = 'agosto'
        case 9:
            mes_to_string = 'septiembre'
        case 10:
            mes_to_string = 'octubre'
        case 11:
            mes_to_string = 'noviembre'
        case 12:
            mes_to_string = 'diciembre'

    return mes_to_string

In [77]:
def get_dfs_from_database(db_mode: str, mes_to_string:str, tolerance:list) -> list[pd.DataFrame]:
    
    if db_mode.lower() == 'local':
    
        db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/comisiones.db'))
        engine = create_engine(f'sqlite:///{db_file_path_str}')

        with engine.connect() as conn, conn.begin():
            ventas = pd.read_sql_table(f'ventas_{mes_to_string}', conn, dtype_backend='numpy_nullable')
            try:
                ultimo_costo = pd.read_sql_table(f'ultimo_costo_{mes_to_string}', conn, dtype_backend='numpy_nullable')
                ultimo_costo['tolerance_order_date'] = ultimo_costo['order_date'].dt.normalize() - pd.Timedelta(days=tolerance[1])
            except:
                ultimo_costo = None

        engine.dispose()
    

    elif db_mode.lower() != 'local':
        raise Exception (f'Sólo existe la base de datos "Local"')


    return ventas, ultimo_costo

In [78]:
def _get_df_from_excel(file_name:str, file_location:str) -> pd.DataFrame:

    file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{file_location}/{file_name}'))
    df = pd.read_excel(file_path_str, dtype_backend='numpy_nullable')

    return df

In [79]:
def prov_locales_df_from_excel() -> list[pd.DataFrame]:
    
    file_name = 'proveedores_oficiales.xlsx'
    file_location = 'compras'

    df = _get_df_from_excel(file_name, file_location)

    prov_oficiales = df.loc[df['oficial'] == 1][['partner_id', 'partner_name']]
    prov_locales = df.loc[df['oficial'] == 0][['partner_id', 'partner_name']]

    return prov_oficiales, prov_locales

In [80]:
def ultimo_costo_sae_df_from_excel(tolerance:list) -> list[pd.DataFrame]:
    
    file_name = 'ultimo_costo_sae.xlsx'
    file_location = 'costo_ventas'

    df = _get_df_from_excel(file_name, file_location)

    df['tolerance_order_date'] = df['order_date'].dt.normalize() - pd.Timedelta(days=tolerance[1])

    return df

In [81]:
def calificados_fase_c_df_from_excel() -> list[pd.DataFrame]:

    file_name = 'calificados_fase_c.xlsx'
    file_location = 'costo_ventas'

    df = _get_df_from_excel(file_name, file_location)

    # Check que verifica que todas las líneas de compra estén calificadas
    if pd.NA in df['calificacion'].unique():
        print('¡Cuidado!, tienes líneas de compras "line_id" sin calificar')
        return None
    
    df['calificacion'] = df['calificacion'].str.upper()
    calificados_fase_c_OK = df.loc[df['calificacion'] == 'OK'] # Líneas 'line_id' que tienen un match con 'fact_line_id'
    calificados_fase_c_B = df.loc[df['calificacion'] == 'B'] # Líneas para regresar a la fase B y que el algoritmo las catalogue.
    calificados_fase_c_C = df.loc[df['calificacion'] == 'C'] # Líneas para regresar a la fase C y que el humano las catalogue.
    calificados_fase_c_D = df.loc[df['calificacion'] == 'D'] # Líneas para que el algoritmo fase D (final) las catalogue.
    calificados_fase_c_E = df.loc[df['calificacion'] == 'E'] # Líneas para eliminar, la observación tiene el por qué.


    return calificados_fase_c_OK, calificados_fase_c_B, calificados_fase_c_C, calificados_fase_c_D, calificados_fase_c_E

In [82]:
def api_call_purchase_doc_func(api_params: dict, search_costo_ventas: list[str] ) -> list[int, dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    purchase_doc_fields = [
                    'name',
                    'state',
                    'partner_id',
                    'partner_ref',
                    'date_approve',
                    'x_fecha_factura',
                    'user_id',
                    'create_uid'
                    ]

    purchase_doc_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'search', [search_costo_ventas])
    purchase_doc_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order', 'read', [purchase_doc_ids], {'fields': purchase_doc_fields})
    
    return purchase_doc_ids, purchase_doc_json

In [83]:
def purchase_doc_func(purchase_doc_json: list[dict], prov_oficiales:pd.DataFrame) -> pd.DataFrame:

    purchase_doc_data = []

    for compra in purchase_doc_json:
        new = {}
        new['order_id'] = compra['id']
        new['order_name'] = compra['name']
        new['order_state'] = compra['state']
        new['order_date'] = compra['date_approve'] if compra['date_approve'] else pd.NA
        new['partner_id'] = compra['partner_id'][0]
        new['partner_name'] = compra['partner_id'][1]
        new['partner_fact_ref'] = compra['partner_ref']
        new['partner_fact_date'] = compra['x_fecha_factura'] if compra['x_fecha_factura'] else pd.NA
        new['capturista'] = compra['create_uid'][1] if compra['create_uid'] else pd.NA
        new['vendedora'] = compra['user_id'][1] if compra['user_id'] else pd.NA

        purchase_doc_data.append(new)

    compras_doc = pd.DataFrame(purchase_doc_data)
    compras_doc['order_date'] = pd.to_datetime(compras_doc['order_date'], format='%Y-%m-%d %H:%M:%S')
    compras_doc['partner_fact_date'] = pd.to_datetime(compras_doc['partner_fact_date'], format='%Y-%m-%d')
    compras_doc['oficial'] = compras_doc['partner_id'].isin(prov_oficiales['partner_id'])

    return compras_doc


In [84]:
def api_call_purchase_line_func(api_params: dict, purchase_doc_ids: list[int]) -> list[dict]:
    
    api_db = api_params['api_db']
    api_clave = api_params['api_clave']
    uid = api_params['api_uid']
    models = api_params['api_models']

    purchase_line_fields = [
                        'order_id',
                        'date_approve',
                        'partner_id',
                        'product_id',
                        'product_qty',
                        'price_unit_discounted'
                        ]

    purchase_line_ids = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'search', [[("order_id.id", "in", purchase_doc_ids)]])
    purchase_line_json = models.execute_kw(api_db, uid, api_clave, 'purchase.order.line', 'read', [purchase_line_ids], {'fields': purchase_line_fields})
    
    return purchase_line_json

In [85]:
def purchase_line_func(purchase_line_json: list[dict]) -> pd.DataFrame:
    
    purchase_line_data = []

    for line in purchase_line_json:
        new = {}
        new['line_id'] = line['id']
        new['order_id'] = line['order_id'][0]
        new['order_name'] = line['order_id'][1]
        new['order_date'] = line['date_approve'] if line['date_approve'] else pd.NA
        new['partner_id'] = line['partner_id'][0]
        new['partner_name'] = line['partner_id'][1]
        new['product_id_pp'] = line['product_id'][0]
        new['product_name'] = line['product_id'][1]
        new['product_qty'] = line['product_qty']
        new['product_cost'] = line['price_unit_discounted']
        
        purchase_line_data.append(new)

    compras_line = pd.DataFrame(purchase_line_data)
    compras_line['order_date'] = pd.to_datetime(compras_line['order_date'], format='%Y-%m-%d %H:%M:%S')

    return compras_line

In [86]:
def compras_func(compras_doc:pd.DataFrame, compras_line:pd.DataFrame, ultimo_costo:pd.DataFrame, tolerance) -> pd.DataFrame:
    
    compras_odoo = pd.merge(
                    compras_line,
                    compras_doc[['order_id', 'partner_fact_ref', 'partner_fact_date', 'capturista', 'vendedora']], 
                    how='left', 
                    on='order_id'
                )

    compras_odoo['tolerance_order_date'] = compras_odoo['order_date'].dt.normalize() - pd.Timedelta(days=tolerance[1])

    cols_to_Int64 = ['line_id', 'order_id', 'partner_id', 'product_id_pp']
    compras_odoo[cols_to_Int64] = compras_odoo[cols_to_Int64].astype('Int64')

    compras_odoo['product_qty'] = compras_odoo['product_qty'].astype('Float64')
    compras_odoo['vendedora'] = compras_odoo['vendedora'].convert_dtypes()

    compras = pd.concat([compras_odoo, ultimo_costo])

    return compras, compras_odoo

# Pruebas

In [87]:
mes = 1
tolerance = [0, 0]

api_params = api_params_func()
search_costo_ventas = search_costo_ventas_func(mes, tolerance)
mes_to_string = mes_to_string_func(mes)

ventas, ultimo_costo = get_dfs_from_database('local', mes_to_string, tolerance)

prov_oficiales, prov_locales = prov_locales_df_from_excel()

if not ultimo_costo:
    ultimo_costo = ultimo_costo_sae_df_from_excel(tolerance)
calificados_fase_c_OK, calificados_fase_c_B, calificados_fase_c_C, calificados_fase_c_D, calificados_fase_c_E = calificados_fase_c_df_from_excel()

purchase_doc_ids, purchase_doc_json = api_call_purchase_doc_func(api_params, search_costo_ventas)
purchase_line_json = api_call_purchase_line_func(api_params, purchase_doc_ids)

compras_doc = purchase_doc_func(purchase_doc_json, prov_oficiales)
compras_line = purchase_line_func(purchase_line_json)
compras, compras_odoo = compras_func(compras_doc, compras_line, ultimo_costo, tolerance)


In [93]:
# Línea para comprobrar que el 100% de los proveedores de Odoo están calificados en la lista de proveedores oficiales

check1 = (compras_doc[~compras_doc['partner_id'].isin(pd.concat([prov_locales, prov_oficiales])['partner_id'])]).drop_duplicates('partner_id')

if not check1.empty:
    print('Hay proveedores no calificados')
    display(check1)

else:
    print('Todo correcto con check1')

Todo correcto con check1


In [90]:
from IPython.display import display

check_costo_venta = (
    pd.merge_asof(
        left = ventas.sort_values('invoice_date'),
        right = compras.sort_values('tolerance_order_date'), 
        
        left_by = 'product_id', 
        right_by = 'product_id_pp', 
        
        left_on = 'invoice_date', 
        right_on = 'tolerance_order_date', 

        direction = 'backward')
)

check2 = check_costo_venta[check_costo_venta['product_cost'].isna()][['product_id', 'product_name_x']]

print(f'Hay {len(check2)} renglones sin costo de la venta.')

if not check2.empty:
    print('Los productos sin costo son los siguientes:')
    display(check2.drop_duplicates('product_id'))

else:
    print('Todo correcto con check2')

Hay 0 renglones sin costo de la venta.
Todo correcto con check2
